In [ ]:
from llm.client import *

llm = LLMClient()
with open("text_test.txt", encoding="utf-8") as file:
    txt = file.read()
output = llm.extract_structured_data(txt)




In [ ]:
# scraping/whatsapp_scraper.py
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


def setup_selenium():
    load_dotenv()
    DRIVER_PATH = os.getenv("DRIVER_PATH")
    CHROME_PROFILE = os.getenv("CHROME_PROFILE")

    options = webdriver.ChromeOptions()
    options.add_argument(f"user-data-dir={CHROME_PROFILE}")
    # options.add_argument("--headless=new")          # 👈 run without opening browser window
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    # driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=options)
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://web.whatsapp.com/")
    return driver

def wait_for_whatsapp_loaded(driver, timeout=600):
    print("Waiting for WhatsApp to load...")
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located((By.ID, "pane-side"))
    )
    print("WhatsApp is ready!")


def search_and_open_chat(driver, chat_name):
    print(f"Searching for chat: {chat_name}")
    search_box = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located(
            (By.XPATH, '//div[@contenteditable="true"][@data-tab="3"]')
        )
    )
    search_box.click()
    search_box.clear()
    search_box.send_keys(chat_name)
    time.sleep(2)  # wait for results to load
    search_box.send_keys(Keys.DOWN)
    search_box.send_keys(Keys.ENTER)
    print(f"Opened chat: {chat_name}")
    time.sleep(3)

def scrape_recent_messages(driver, limit=20):
    print("Scraping recent messages...")

    # Find all message bubbles (incoming & outgoing)
    bubbles = driver.find_elements(
        By.XPATH, "//div[contains(@class,'message-in') or contains(@class,'message-out')]"
    )
    
    if not bubbles:
        raise RuntimeError("No messages found — chat may not have loaded yet.")
    results = []
    for msg in bubbles[-limit:]:
        sender = "You" if "message-out" in msg.get_attribute("class") else "Contact"

        # Try all possible selectors for text
        message_text = msg.text


        if not message_text:
            message_text = "<media omitted>"

        # Extract time (new DOM uses <span data-testid='msg-meta'>)
        try:
            msg_time = re.findall(r'\b(?:[01]\d|2[0-3]):[0-5]\d\b', message_text)[0]
        except Exception:
            msg_time = ""

        results.append(
            {"sender": sender, "time": msg_time, "message": message_text}
        )
    return results

    

driver = setup_selenium()
wait_for_whatsapp_loaded(driver)

chat_name = "Modon & First Step"
search_and_open_chat(driver, chat_name)
messages = scrape_recent_messages(driver, limit=20)

Waiting for WhatsApp to load...
WhatsApp is ready!
Searching for chat: Modon & First Step
Opened chat: Modon & First Step
Scraping recent messages...


In [39]:
messages[3]

{'sender': 'Contact',
 'time': ['13:06'],
 'message': 'Mostafa Brekaa\n+20 10 69362010\n282 kB\nLagoons New Capital \n شقق 3 غرف مساحه  147متر\nالسعر يبدأ من 4,300,000 جنيه\n شقق 3 غرف +  ليفينج + ريسبشن 3 قطع مساحه  186متر\nالسعر يبدأ من 5,400,000 جنيه\n دوبلكس مساحه 252 متر + 53 متر جاردن \nالسعر يبدأ من 8,000,000 جنيه\n تاون هاوس مساحه 185 متر\nالسعر يبدأ من 12,800,000 جنيه\n توين هاوس مساحه 215متر\nالسعر يبدأ من 16,800,000 جنيه\nانظمه السداد :\n10% over 8 years\nالصيانه 10%\n\u2060طوب احمر\n\u2060الاستلام ٣ سنوات\n13:06'}